<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/Experiment_ask/fine_tuning/pubmed/askApatient_folder0_pubmed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

In [0]:
base_dir = 'medical-concept-normalization'

In [0]:
os.mkdir(base_dir)

In [0]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [0]:
!curl https://course-v3.fast.ai/setup/colab | bash

In [0]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

In [0]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [0]:
path = Path('medical-concept-normalization/data_collection')

In [0]:
# training data set

train_csv = path/'AskAPatient.fold-0.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [0]:
# validating data set

valid = pd.read_csv(path/"AskAPatient.fold-0.validation.csv",header=None)
valid.head()

In [0]:
# testing data set

testing = pd.read_csv(path/"AskAPatient.fold-0.testing.csv",header=None)
testing.head()

In [0]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'fine-tuning_language_model_cadec_data.csv')

In [0]:
# Creating DataBunch for classification

data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_lm.save()
data_clas.save()

In [0]:
data_lm.train_ds.vocab.itos

In [0]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [0]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [0]:
learn.fit_one_cycle(1, 1e-2)

In [0]:
learn.fit_one_cycle(1, 1e-2)

In [0]:
learn.fit_one_cycle(1, 1e-3)

In [0]:
learn.fit_one_cycle(1, 1e-3)

In [0]:
learn.save_encoder('ft_fold0_first')

In [0]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('ft_fold0_first')
learn.fit_one_cycle(1, 1e-2)

In [0]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

In [0]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

In [0]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

In [0]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

In [0]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

In [0]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

In [0]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

In [0]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

In [0]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

In [0]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

In [0]:
learn.data = data_class
learn.get_preds(ds_type = DatasetType.Test, ordered=True)